In [ ]:
import json

def BIO_labels(raw_words, aspects):
    words = raw_words.split()
    labels = ['O'] * len(words)
    for aspect in aspects:
        labels[aspect['from']:aspect['to']] = ['B'] + ['I'] * (aspect['to'] - aspect['from'] - 1)
    return labels

with open('Laptop_Review_Test.json', 'r') as file:
    data = json.load(file)

processed_data = {}

for i, entry in enumerate(data):
    processed_data[str(i+1)] = {'text': entry['raw_words'], 'labels': BIO_labels(entry['raw_words'], entry['aspects'])}

with open('Laptop_Review_Test_BIO.json', 'w') as file:
    json.dump(processed_data, file, indent=4)


In [ ]:
wr = "In the shop , these MacBooks are encased in a soft rubber enclosure - so you will never know about the razor edge until you buy it , get it home , break the seal and use it ( very clever con ) ."
print(wr.split())

['In', 'the', 'shop', ',', 'these', 'MacBooks', 'are', 'encased', 'in', 'a', 'soft', 'rubber', 'enclosure', '-', 'so', 'you', 'will', 'never', 'know', 'about', 'the', 'razor', 'edge', 'until', 'you', 'buy', 'it', ',', 'get', 'it', 'home', ',', 'break', 'the', 'seal', 'and', 'use', 'it', '(', 'very', 'clever', 'con', ')', '.']


In [1]:
##PART1A STRATIFIED SPLIT

import json
from sklearn.model_selection import train_test_split

with open('NER_TRAIN_JUDGEMENT.json', 'r') as f:
    data = json.load(f)

filtered_data = [item for item in data if item.get('annotations') and item['annotations'][0].get('result')]

labels = []
for item in filtered_data:

    label = item['annotations'][0]['result'][0]['value']['labels'][0]
    labels.append(label)

train_data, test_data = train_test_split(filtered_data, test_size=0.15, stratify=labels, random_state=42)

with open('NER_TRAIN_JUDGEMENT_SPLIT.json', 'w') as f_train:
    json.dump(train_data, f_train, indent=4)

with open('NER_VAL_JUDGEMENT_SPLIT.json', 'w') as f_val:
    json.dump(test_data, f_val, indent=4)


In [ ]:
# model = Sequential()
# model.add(embedding_layer)

# model.add(LSTM(EMBEDDING_DIM, dropout=0.2, recurrent_dropout=0.2))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'

In [ ]:
#final code #part 1A #for the given example there must be 16 tokens idk how #rest everything is perfect
#FIXED BUT GOTTA CHECK ONCE AGAIN

import json

# Load the dataset
with open('NER_TEST_JUDGEMENT.json', 'r') as f:
    dataset = json.load(f)

# Define the legal entities
legal_entities = ["JUDGE", "PERSON", "ORGANIZATION", "LOCATION", "DATE", "TIME", "MONEY", "PERCENT", "LAW", "CRIME", "ORDINAL", "QUANTITY", "CARDINAL"]

# Function to assign BIO labels to tokens
def bio_labels(text, annotations):
    tokens = text.split(" ")
    labels = ['O'] * len(tokens)  # Initialize all labels as 'O'

    for annotation in annotations:
        for result in annotation['result']:
            start = result['value']['start']
            end = result['value']['end']
            label = result['value']['labels'][0]
            entity_text = text[start:end]

            entity_tokens = entity_text.split()
            token_index = 0
            for i, token in enumerate(tokens):
                if token_index >= len(entity_tokens):
                    break
                if entity_tokens[token_index] in token:
                    labels[i] = 'B_' + label if token_index == 0 else 'I_' + label
                    token_index += 1

    return labels

# Process each data entry and assign BIO labels
processed_data = {entry['id']: {'text': entry['data']['text'], 'labels': bio_labels(entry['data']['text'], entry['annotations'])} for entry in dataset}

# Save processed data to a new JSON file
with open('NER_TEST.json', 'w') as f:
    json.dump(processed_data, f, indent=4)


In [ ]:
#code with ID fixedd #part 1A

import json

with open('NER_TRAIN_JUDGEMENT_SPLIT.json', 'r') as f:
    dataset = json.load(f)

legal_entities = ["JUDGE", "PERSON", "ORGANIZATION", "LOCATION", "DATE", "TIME", "MONEY", "PERCENT", "LAW", "CRIME", "ORDINAL", "QUANTITY", "CARDINAL"]

def bio_labels(text, annotations):
    tokens = text.split()
    labels = ['O'] * len(tokens)

    for annotation in annotations:
        for result in annotation['result']:
            start, end = result['value']['start'], result['value']['end']
            label = result['value']['labels'][0]
            entity_tokens = text[start:end].split()
            for i, token in enumerate(tokens):
                if any(entity_token in token for entity_token in entity_tokens):
                    labels[i] = 'B_' + label if i == tokens.index(entity_tokens[0]) else 'I_' + label

    return labels

processed_data = [{'id': entry['id'], 'text': entry['data']['text'], 'labels': bio_labels(entry['data']['text'], entry['annotations'])} for entry in dataset]

with open('NER_TRAIN.json', 'w') as f:
    json.dump(processed_data, f, indent=4)
